In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Set up data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
# Load data from the directories
train_data_dir = 'C:/Users/USER/Desktop/Emotion Recognition/datasets/face_dataset/train'
test_data_dir = 'C:/Users/USER/Desktop/Emotion Recognition/datasets/face_dataset/test'

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # ResNet50 input size
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # ResNet50 input size
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    shuffle=False
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Build the Model Using ResNet50

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Load the ResNet50 model pre-trained on ImageNet, without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 (pre-trained weights)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for emotion classification
model = models.Sequential()
model.add(base_model)  # Add the ResNet50 base model
model.add(layers.GlobalAveragePooling2D())  # Global average pooling
model.add(layers.Dense(512, activation='relu'))  # Fully connected layer
model.add(layers.Dropout(0.5))  # Dropout for regularization
model.add(layers.Dense(7, activation='softmax'))  # 7 emotions, softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,640,391 (94.00 MB)

 Trainable params: 1,052,679 (4.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [5]:
# Training the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,  # Adjust epochs as needed
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=test_generator.samples // test_generator.batch_size
)

C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2248 - loss: 1.8834

C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 1727s 2s/step - accuracy: 0.2248 - loss: 1.8834 - val_accuracy: 0.2478 - val_loss: 1.8108
Epoch 2/10
  1/897 ━━━━━━━━━━━━━━━━━━━━ 22:21 1s/step - accuracy: 0.3125 - loss: 1.9257

C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.3125 - loss: 1.9257 - val_accuracy: 0.0000e+00 - val_loss: 2.2444
Epoch 3/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2510 - loss: 1.8221

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Save the trained model
model.save('C:/Users/USER/Desktop/Emotion Recognition/backend/models/face_emotion_modelResNet.h5')

### Predict Emotions

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Function to preprocess image for prediction
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (224, 224))  # Resize to 224x224
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    return img

# Function to predict emotion
def predict_emotion(img_path):
    img = preprocess_image(img_path)
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)
    class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    print(f"Predicted Emotion: {class_labels[predicted_class[0]]}")
    return class_labels[predicted_class[0]]

# Example image path
img_path = 'C:/Users/USER/Desktop/Emotion Recognition/datasets/face_dataset/train/happy/Training_185749.jpg'

# Predict the emotion
predicted_emotion = predict_emotion(img_path)
